In [ ]:
#| default_exp agent

In [ ]:
#| export
import os
from typing import List, Tuple, Union, Dict
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, \
    ChatPromptTemplate
from pino_inferior.models import aengine
from pino_inferior.core import PROMPTS_DIR, OPENAI_API_KEY, VECTOR_DB, VECTOR_DB_PARAMS, MEMORY_PARAMS
from pino_inferior.message import Message
from pino_inferior.memory import Memory, INPUT_RETRIEVER_QUERY, OUTPUT_RETRIEVER_DOCUMENTS
from langchain.schema.runnable import RunnableSequence
from langchain.chains import TransformChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from dataclasses import dataclass
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage
from langchain.prompts.chat import ChatPromptValue
from pino_inferior.fallacy import build_fallacy_detection_chain, read_fallacies, FALLACIES_FNAME, \
    INPUT_QUERY as INPUT_FALLACY_QUERY, OUTPUT_SHORT_ANSWER as OUTPUT_FALLACY_QUERY
from datetime import datetime
from langchain.chat_models.base import BaseChatModel
from enum import Enum
from typing import Callable
import asyncio
import tiktoken
from datetime import datetime
from dataclasses import dataclass

In [ ]:
#| export
AGENT_PROMPTS_DIR = os.path.join(PROMPTS_DIR, "roleplay_agent")
TOOLS_PROMPTS_DIR = os.path.join(AGENT_PROMPTS_DIR, "tools")

In [ ]:
#| export
def _read_file(fname: str) -> str:
    with open(fname, "r", encoding="utf-8") as src:
        return src.read()

## Implementation

In [ ]:
#| export
AGENT_INPUT_HISTORY = "history"
AGENT_INPUT_TOOLS = "tools"
AGENT_INPUT_CONTEXT = "context"
AGENT_INPUT_FALLACIES = "fallacies"
AGENT_INPUT_USERNAME = "name"
AGENT_INPUT_CHARACTER = "character"
AGENT_INPUT_GOAL = "goals"
AGENT_INPUT_TIME = "time"
AGENT_INPUT_STYLE_EXAMPLES = "style_examples"
AGENT_INPUT_STYLE_DESCRIPTION = "style_description"

AGENT_INTERMEDIATE_HISTORY_STR = "input_str"
AGENT_INTERMEDIATE_TOOLS_STR = "tools_str"
AGENT_INTERMEDIATE_TIME_STR = "time_str"
AGENT_INTERMEDIATE_STYLE_EXAMPLES = "style_examples_str"

### Tool representations

In [ ]:
#| export
@dataclass
class ToolDescription:
    name: str
    description: str
    input_key: Union[str, None]
    output_key: str

### Preprocessing

In [ ]:
#| export
agent_system_prompt = SystemMessagePromptTemplate.from_template(_read_file(
    os.path.join(AGENT_PROMPTS_DIR, "system.txt")
))
agent_instruction_prompt = HumanMessagePromptTemplate.from_template(_read_file(
    os.path.join(AGENT_PROMPTS_DIR, "instruction.txt")
))
agent_llm_prompt = ChatPromptTemplate.from_messages([agent_system_prompt, agent_instruction_prompt])

In [ ]:
#| export
def build_stringification_chain(
        length_function: Callable[[str], int],
        max_messages_length: int,
        max_tools_length: int,
        max_context_length: int,
        max_username_length: int,
        max_character_length: int,
        max_goal_length: int,
        max_style_examples_length: int,
        max_style_description_length: int,
) -> TransformChain:
    def _stringify_messages(row):
        messages: List[Message] = row[AGENT_INPUT_HISTORY]
        while True:
            messages_str = "\n\n".join(map(str, messages))
            if length_function(messages_str) <= max_messages_length:
                break
            else:
                messages = messages[1:]
        assert len(messages) > 0, \
            f"Only after cutting all the messages total length become less than {max_messages_length}"
        return messages_str
    
    def _stringify_tools(row):
        tools: List[Tuple[ToolDescription, RunnableSequence]] = row[AGENT_INPUT_TOOLS]
        tools_str = "\n\n".join([
            f"-- {tool.name}: {tool.description}"
            for tool, _ in tools
        ])
        assert length_function(tools_str) <= max_tools_length, \
            f"Total size of tools description should be lower than {max_tools_length}"
        return tools_str
        
    def _stringify_time(row):
        time: datetime = row[AGENT_INPUT_TIME]
        return time.strftime("%d %b %Y %H:%M")
    
    def _stringify_style_examples(row):
        examples: List[str] = row[AGENT_INPUT_STYLE_EXAMPLES]
        examples_str = "\n\n".join(examples)
        assert length_function(examples_str) <= max_style_examples_length, \
            f"Total size of style exampes should be lower than {max_style_examples_length}"
        return examples_str    

    def agent_stringify(row):
        messages_str = _stringify_messages(row)
        tools_str = _stringify_tools(row)
        time_str = _stringify_time(row)
        style_examples_str = _stringify_style_examples(row)
        
        assert length_function(row[AGENT_INPUT_CONTEXT]) <= max_context_length
        assert length_function(row[AGENT_INPUT_USERNAME]) <= max_username_length
        assert length_function(row[AGENT_INPUT_CHARACTER]) <= max_character_length
        assert length_function(row[AGENT_INPUT_GOAL]) <= max_goal_length
        assert length_function(row[AGENT_INPUT_STYLE_DESCRIPTION]) <= max_style_description_length

        return {
            AGENT_INTERMEDIATE_HISTORY_STR: messages_str,
            AGENT_INTERMEDIATE_TOOLS_STR: tools_str,
            AGENT_INTERMEDIATE_TIME_STR: time_str,
            AGENT_INTERMEDIATE_STYLE_EXAMPLES: style_examples_str
        }
    
    async def aagent_stringify(row):
        return agent_stringify(row)
    
    return TransformChain(
        transform=agent_stringify,
        atransform=aagent_stringify,
        input_variables=[
            AGENT_INPUT_HISTORY,
            AGENT_INPUT_TOOLS,
            AGENT_INPUT_CONTEXT,
            AGENT_INPUT_USERNAME,
            AGENT_INPUT_CHARACTER,
            AGENT_INPUT_GOAL,
            AGENT_INPUT_TIME,
            AGENT_INPUT_STYLE_EXAMPLES,
            AGENT_INPUT_STYLE_DESCRIPTION,
        ],
        output_variables=[
            AGENT_INTERMEDIATE_HISTORY_STR,
            AGENT_INTERMEDIATE_TOOLS_STR,
            AGENT_INTERMEDIATE_TIME_STR,
            AGENT_INTERMEDIATE_STYLE_EXAMPLES,
        ],
    )

### Agent loop

#### LLM call

In [ ]:
#| export
async def _arun_agent_llm(agent_prompt: ChatPromptTemplate,
                          agent_llm: BaseChatModel,
                          tool_call_stop_sequence: str,
                          response_stop_sequence: str) -> str:
    response = await agent_llm.ainvoke(
        agent_prompt,
        stop=[tool_call_stop_sequence, response_stop_sequence]
    )
    return response.content

#### LLM output parsing

In [ ]:
#| export
def _split_by_marker(text: str, open_marker: str, close_marker: str) -> List[str]:
    blocks = text.split(open_marker)
    before_last_open_marker = open_marker.join(blocks[:-1])
    before_last_close_marker = blocks[-1].split(close_marker)[0]
    return before_last_open_marker, before_last_close_marker

In [ ]:
#| export
class _NextAction:
    TOOL = 1
    RESPONSE = 2
    UNKNOWN = 3


@dataclass
class _ParsedResponse:
    chain_of_thoughts: str
    next_action: _NextAction
    next_action_type: str
    next_action_query: str

In [ ]:
#| export
class LLMOutputParseError(ValueError):
    def __init__(self, output: str):
        super(LLMOutputParseError, self).__init__(
            f"LLM output parsing error: {output}"
        )

In [ ]:
#| export
def _parse_agent_output(response: str, tools: List[ToolDescription], response_marker: str) -> _ParsedResponse:
    for tool in tools:
        tool_open_marker = f"[{tool.name}]"
        tool_close_marker = f"[/{tool.name}]"
        if response.endswith(tool_close_marker):
            chain_of_thoughts, query = _split_by_marker(response,
                                                        tool_open_marker,
                                                        tool_close_marker)
            return _ParsedResponse(
                chain_of_thoughts=chain_of_thoughts,
                next_action=_NextAction.TOOL,
                next_action_type=tool.name,
                next_action_query=query
            )
    response_open_marker = f"[{response_marker}]"
    response_close_marker = f"[/{response_marker}]"
    if response_open_marker in response:
        chain_of_thoughts, response = _split_by_marker(response,
                                                       response_open_marker,
                                                       response_close_marker)
        return _ParsedResponse(
            chain_of_thoughts=chain_of_thoughts,
            next_action=_NextAction.RESPONSE,
            next_action_type="",
            next_action_query=response,
        )
    raise LLMOutputParseError(response)

In [ ]:
#| export
def _extract_tool_representations(tools: List[Tuple[ToolDescription, RunnableSequence]]) -> Tuple[List[ToolDescription], Dict[str, Tuple[ToolDescription, RunnableSequence]]]:
    tool_descriptions = [
        description
        for description, _ in tools
    ]
    tools_by_name = {
        description.name: (description, tool)
        for description, tool in tools
    }
    return tool_descriptions, tools_by_name


def _process_tool(inputs: dict, response: _NextAction, tools_by_name: Dict[str, Tuple[ToolDescription, RunnableSequence]]) \
    -> Tuple[dict, RunnableSequence, str, str]:
    assert response.next_action_type in tools_by_name
    tool_description, tool_chain = tools_by_name[response.next_action_type]
    tool_inputs = dict(inputs, **{tool_description.input_key: response.next_action_query})
    return tool_inputs, tool_chain, tool_description.output_key, tool_description.name


async def _process_tool_response(tool_name: str,
                                 query: str,
                                 output: str,
                                 tool_call_stop_sequence: str,
                                 tool_call_close_sequence: str,
                                 tool_length_function: Callable[[str], int],
                                 tool_cut_function: Callable[[str, int], str],
                                 tool_query_max_length: int,
                                 tool_response_max_length: int) \
                                    -> Tuple[str, None, bool]:
    if tool_length_function(query) > tool_query_max_length:
        query = tool_cut_function(query, tool_query_max_length)
    if tool_length_function(output) > tool_response_max_length:
        output = tool_cut_function(output, tool_response_max_length)
    suffix = f"[{tool_name}]{query}[/{tool_name}]" + \
        f"{tool_call_stop_sequence}\n" + \
        f"```\n{output}\n```\n" + \
        f"{tool_call_close_sequence}"
    final_response = None
    continue_further = True
    return suffix, final_response, continue_further


async def _aprocess_agent_iteration_output(
        inputs: dict,
        llm_output: str,
        tool_call_stop_sequence: str,
        tool_call_close_sequence: str,
        tools: List[Tuple[ToolDescription, RunnableSequence]],
        tool_length_function: Callable[[str], int],
        tool_cut_function: Callable[[str, int], str],
        tool_query_max_length: int,
        tool_response_max_length: int,
        response_marker: str,
) -> Tuple[AIMessage, Union[str, None], bool]:
    tool_descriptions, tools_by_name = _extract_tool_representations(tools)
    response = _parse_agent_output(llm_output, tool_descriptions, response_marker)
    assert response.next_action in {_NextAction.TOOL, _NextAction.RESPONSE}
    if response.next_action == _NextAction.TOOL:
        tool_inputs, tool_chain, tool_output_key, tool_name = _process_tool(inputs, response, tools_by_name)
        tool_output = (await tool_chain.ainvoke(tool_inputs))[tool_output_key]
        suffix, final_response, continue_further = await _process_tool_response(
            tool_name,
            response.next_action_query,
            tool_output,
            tool_call_stop_sequence,
            tool_call_close_sequence,
            tool_length_function,
            tool_cut_function,
            tool_query_max_length,
            tool_response_max_length,
        )
    elif response.next_action == _NextAction.RESPONSE:
        suffix = f"[{response_marker}]{response.next_action_query}[/{response_marker}]"
        final_response = response.next_action_query
        continue_further = False
    message = AIMessage(content=f"{response.chain_of_thoughts}{suffix}")
    return message, final_response, continue_further

#### Running one iteration

In [ ]:
#| export
async def _arun_agent_iteration(inputs: dict,
                         agent_prompt: ChatPromptValue,
                         agent_llm: BaseChatModel,
                         tool_call_stop_sequence: str,
                         tool_call_close_sequence: str,
                         tools: List[Tuple[ToolDescription, RunnableSequence]],
                         tool_length_function: Callable[[str], int],
                         tool_cut_function: Callable[[str, int], str],
                         tool_query_max_length: int,
                         tool_response_max_length: int,
                         response_marker: str) -> Tuple[AIMessage, Union[str, None], bool]:
    llm_output = await _arun_agent_llm(
        agent_prompt=agent_prompt,
        agent_llm=agent_llm,
        tool_call_stop_sequence=tool_call_stop_sequence,
        response_stop_sequence=f"[/{response_marker}]",
    )
    return await _aprocess_agent_iteration_output(
        inputs,
        llm_output,
        tool_call_stop_sequence,
        tool_call_close_sequence,
        tools,
        tool_length_function,
        tool_cut_function,
        tool_query_max_length,
        tool_response_max_length,
        response_marker,
    )

#### Agent main loop

In [ ]:
#| export
def run_agent(inputs: dict,
              agent_input_preprocessor: RunnableSequence,
              agent_llm: BaseChatModel,
              tool_call_stop_sequence: str,
              tool_call_close_sequence: str,
              tools: List[Tuple[ToolDescription, RunnableSequence]],
              tool_length_function: Callable[[str], int],
              tool_cut_function: Callable[[str, int], str],
              tool_query_max_length: int,
              tool_response_max_length: int,
              response_marker: str,
              max_iteration_count: int,
              max_token_count: int) -> str:
    return asyncio.get_event_loop().run_until_complete(
        arun_agent(
            inputs,
            agent_input_preprocessor,
            agent_llm,
            tool_call_stop_sequence,
            tool_call_close_sequence,
            tools,
            tool_length_function,
            tool_cut_function,
            tool_query_max_length,
            tool_response_max_length,
            response_marker,
            max_iteration_count,
            max_token_count,
        )
    )

    
async def arun_agent(inputs: dict,
                     agent_input_preprocessor: RunnableSequence,
                     agent_llm: BaseChatModel,
                     tool_call_stop_sequence: str,
                     tool_call_close_sequence: str,
                     tools: List[Tuple[ToolDescription, RunnableSequence]],
                     tool_length_function: Callable[[str], int],
                     tool_cut_function: Callable[[str, int], str],
                     tool_query_max_length: int,
                     tool_response_max_length: int,
                     response_marker: str,
                     max_iteration_count: int,
                     max_token_count: int) -> str:
    chat_inputs: ChatPromptValue = await agent_input_preprocessor.ainvoke(inputs)
    ai_messages = []
    for _ in range(max_iteration_count):
        token_count_without_ai = agent_llm.get_num_tokens_from_messages(chat_inputs.messages)
        assert token_count_without_ai <= max_token_count, \
            f"Session length ({token_count_without_ai}) exceeds {max_iteration_count} limit"

        ai_message, final_response, continue_further = await _arun_agent_iteration(
            inputs,
            chat_inputs,
            agent_llm,
            tool_call_stop_sequence,
            tool_call_close_sequence,
            tools,
            tool_length_function,
            tool_cut_function,
            tool_query_max_length,
            tool_response_max_length,
            response_marker
        )
        ai_messages.append(ai_message)
        chat_inputs.messages.append(ai_message)

        if continue_further:
            token_count_with_ai = agent_llm.get_num_tokens_from_messages(chat_inputs.messages)
            assert token_count_with_ai <= max_token_count, \
                f"With latest AI message session length ({token_count_with_ai}) exceeds {max_token_count} while no response achieved"
            continue
        assert final_response is not None, "Did not got final response"
        return final_response
    raise ValueError(f"Did not got final LLM response after {max_iteration_count} iterations")

### Agent wrapper

In [ ]:
# export
@dataclass
class LengthConfig:
    cut_function: Callable[[str, int], str]
    length_function: Callable[[str], int]
    max_messages_length: int = 2048
    max_tools_length: int = 256
    max_context_length: int = 512
    max_username_length: int = 10
    max_character_length: int = 256
    max_goal_length: int = 256
    max_style_examples_length: int = 512
    max_style_description_length: int = 512
    max_tool_query_length: int = 32
    max_tool_response_length: int = 512
    max_session_length: int = 8 * 1024


@dataclass
class PromptMarkupConfig:
    tool_call_stop_sequence: str = "[call]"
    tool_call_close_sequence: str = "[/call]"
    response_marker = "response"


class RolePlayAgent:
    def __init__(self,
                 tools: List[Tuple[ToolDescription, RunnableSequence]],
                 lengths: LengthConfig,
                 prompt_markup: PromptMarkupConfig,
                 llm: BaseChatModel,
                 max_iter: int = 5) -> None:
        self.tools = tools
        self.lengths = lengths
        self.prompt_markup = prompt_markup
        self.llm = llm
        self.preprocessing_chain = self._build_preprocessing_chain()
        self.max_iter = max_iter

    def _build_preprocessing_chain(self) -> RunnableSequence:
        agent_stringify_transform = build_stringification_chain(
            length_function=self.lengths.length_function,
            max_messages_length=self.lengths.max_messages_length,
            max_tools_length=self.lengths.max_tools_length,
            max_context_length=self.lengths.max_context_length,
            max_username_length=self.lengths.max_username_length,
            max_character_length=self.lengths.max_character_length,
            max_goal_length=self.lengths.max_goal_length,
            max_style_examples_length=self.lengths.max_style_examples_length,
            max_style_description_length=self.lengths.max_style_description_length,
        )
        agent_preprocessing_chain = agent_stringify_transform | agent_llm_prompt
        return agent_preprocessing_chain
    
    async def arun(self, inputs: dict) -> str:
        return await arun_agent(
            inputs=inputs,
            agent_input_preprocessor=self.preprocessing_chain,
            agent_llm=self.llm,
            tool_call_stop_sequence=self.prompt_markup.tool_call_stop_sequence,
            tool_call_close_sequence=self.prompt_markup.tool_call_close_sequence,
            tools=self.tools,
            tool_length_function=self.lengths.length_function,
            tool_cut_function=self.lengths.cut_function,
            tool_query_max_length=self.lengths.max_tool_query_length,
            tool_response_max_length=self.lengths.max_tool_response_length,
            response_marker=self.prompt_markup.response_marker,
            max_iteration_count=self.max_iter,
            max_token_count=self.lengths.max_session_length,
        )
    
    def run(self, inputs: dict) -> str:
        return run_agent(
            inputs=inputs,
            agent_input_preprocessor=self.preprocessing_chain,
            agent_llm=self.llm,
            tool_call_stop_sequence=self.prompt_markup.tool_call_stop_sequence,
            tool_call_close_sequence=self.prompt_markup.tool_call_close_sequence,
            tools=self.tools,
            tool_length_function=self.lengths.length_function,
            tool_cut_function=self.lengths.cut_function,
            tool_query_max_length=self.lengths.max_tool_query_length,
            tool_response_max_length=self.lengths.max_tool_response_length,
            response_marker=self.prompt_markup.response_marker,
            max_iteration_count=self.max_iter,
            max_token_count=self.lengths.max_session_length,
        )

### Agent example

#### Tools

In [ ]:
sentence_encoder = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model="text-embedding-ada-002",
)
memory_container = Memory(
    engine=aengine,
    vector_db=VECTOR_DB(
        embedding_function=sentence_encoder,
        **VECTOR_DB_PARAMS
    ),
    **MEMORY_PARAMS
)
memory_tool = (
    ToolDescription(
        name="memory",
        description=_read_file(os.path.join(TOOLS_PROMPTS_DIR, "memory.txt")),
        input_key=INPUT_RETRIEVER_QUERY,
        output_key=OUTPUT_RETRIEVER_DOCUMENTS,
    ),
    memory_container.build_retriever_chain()
)
memory_tool

(ToolDescription(name='memory', description='External memory tool. \nYour training data is limited, but you can use it.\nQuery should be exact since memory will only see it, not the surrounding context.\nSometimes memory might return non-relevant results (like sudden associations outside the discussed context).\nReturns retrieved documents.\nCall it this way: [memory]%Your query[/memory][call]', input_key='query', output_key='documents_text'),
 TransformChain(input_variables=['query'], output_variables=['documents'], transform_cb=<function Memory.build_retriever_chain.<locals>._retrieve_documents>, atransform_cb=<function Memory.build_retriever_chain.<locals>._aretrieve_documents>)
 | TransformChain(input_variables=['documents'], output_variables=['documents_text'], transform_cb=<function Memory.build_retriever_chain.<locals>._stringify_documents>, atransform_cb=<function Memory.build_retriever_chain.<locals>._astringify_documents>))

In [ ]:
fallacy_llm = ChatOpenAI(
    model_name="gpt-4-0613",
    openai_api_key=OPENAI_API_KEY,
    streaming=True,
)
fallacy_tool = (
    ToolDescription(
        name="fallacy",
        description=_read_file(os.path.join(TOOLS_PROMPTS_DIR, "fallacy.txt")),
        input_key=INPUT_FALLACY_QUERY,
        output_key=OUTPUT_FALLACY_QUERY,
    ),
    build_fallacy_detection_chain(fallacy_llm)
)
fallacy_tool.__class__

tuple

#### Initialization

In [ ]:
agent_llm = ChatOpenAI(
    model_name="gpt-4-0613",
    openai_api_key=OPENAI_API_KEY,
    streaming=True,
)
agent_llm_encoding = tiktoken.encoding_for_model(agent_llm.model_name)

In [ ]:
agent = RolePlayAgent(
    tools=[memory_tool, fallacy_tool],
    lengths=LengthConfig(
        cut_function=lambda text, length: agent_llm_encoding.decode(agent_llm_encoding.encode(text)[:length]),
        length_function=lambda text: len(agent_llm_encoding.encode(text)),
    ),
    prompt_markup=PromptMarkupConfig(),
    llm=agent_llm,
    max_iter=5,
)

#### Running

In [ ]:
inputs = {
    AGENT_INPUT_TIME: datetime.now(),
    AGENT_INPUT_CONTEXT: "Пост о войне России и Украины",
    AGENT_INPUT_FALLACIES: read_fallacies(FALLACIES_FNAME),
    AGENT_INPUT_HISTORY: [
        Message("Moonlight", datetime.now(),
                "Мы скоро закончим с Украиной."),
        Message("alex4321", datetime.now(),
                "А что, случился какой-то прогресс после 6 месяцев взятия Бахмута?\n\n" + \
          "Ну, чтобы подозревать что это произойдёт вскоре, " + \
                    "а не затянется на годы независимо от исхода."),
        Message("Moonlight", datetime.now(),
                "Время - ресурс, у нас его дохуя.")
    ],
    AGENT_INPUT_TOOLS: [
        fallacy_tool,
        memory_tool,
    ],
    AGENT_INPUT_USERNAME: "alex4321",
    AGENT_INPUT_CHARACTER: "you are a programmer, 29 y.o. male",
    AGENT_INPUT_GOAL: "Convince as much people as possible that the ongoing war between Russia and Ukraine is only bringing more problem, that it do not solve any issues, and moreover - break the impression that it is widely supported. To do so - express your opinion, and debunk opponents opinions.",
    AGENT_INPUT_STYLE_EXAMPLES: [
        "В вакууме, да :Yoba:. Оба же тут существовали всё время или имели таки достигнутое соглашение, чтобы было от чего отталкиваться как опорной точки :Yoba:",
        "Не особо-то может. Не привлекая население в виде не 1% принудительного мармелада и пары процентов добровольного, а в виде процентов 10.\nА то, чтобы привлечь большое количество - неплохо бы, чтобы они понимали, нахуя это им надо. А то так численность военкомов может начать неприемлемо быстро падать, а следом их желание работать.\nА с этим у пропаганды проблема. Вот с чем у них нет проблем, так это с стимуляцией пассивности, но это обратно нужному (для названной вами задачи).\nДа и опять же - ну вот убедишь ты в идее не какого-нибудь Стрелкова и клуб рассерженных долбоёбов, а большое число людей. Что делать, когда (не если, а когда) идея станет неактуальной? Показательной посадкой пары человек дело не закончится же.\n",
        "Точнее не так - смену она не устраивала.\nОна просто выстрелила себе в ногу так, что потом что-то новое приходилось строить не апгрейдом предыдущей системы, а из кусков её трупа.",
    ],
    AGENT_INPUT_STYLE_DESCRIPTION: "- Non-formal style, using mainly Russian language " + \
        "(my English is a bit screwed up)" + \
        "\n- Brief. Most time.\n" + \
        "- Overuse memes sometimes.",
}

In [ ]:
print(agent.run(inputs))

Время - это, конечно, ресурс, но его наличие не гарантирует успех. Если не учесть такие факторы как стратегия, доступные ресурсы или международные отношения, то можно попасть в ловушку логической ошибки ad Ignorantiam. Так что не стоит полагаться только на время, думаю, этот конфликт требует более комплексного подхода.


In [ ]:
print(await agent.arun(inputs))

Ты говоришь, что у нас время - ресурс, и его дохуя. Но ведь это ж подход как в анекдоте: "У нас бананы растут? - Нет. - А почему? - А нихуя. - Вот и я о том же".  Это Argumentum ad Ignorantiam, братан. Без конкретики и доказательств это просто пустой звук.

Ну и второе, война - это не только ресурсы в виде времени. Это и люди, которые тратят свои жизни, здоровье, семьи. И чем дольше война, тем больше такой цены приходится платить.

А уж про то, что все закончится быстро - это уже вообще Hasty Generalization. Никто не знает, как все сложится, и говорить о скором окончании - это как стрелять в темную. 

Не думай, что я против России, нет, просто надо понимать, что война это не решение проблем, а их усугубление. 


In [ ]:
import nbdev; nbdev.nbdev_export()